# The Heavy-Tail Phenomenon in SGD

This notebook contains the code for reproducing the synthetic data experiments reported in the paper. 

The current code handles the case where the step-size $\eta$ and the batch-size $b$ are varying; however, it is straightforward to modify the code for varying dimension $d$ and curvature $\sigma$.

For the neural network experiments, we used the code provided in 
U. Simsekli, L. Sagun, M. Gurbuzbalaban, "A Tail-Index Analysis of Stochastic Gradient Noise in Deep Neural Networks", ICML 2019. We are also submitting our version for convenience. 

In [ ]:
# Required libraries are listed below. 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import levy_stable
import scipy.io as sio
import math
import levy
import powerlaw 
from joblib import Parallel, delayed
import torch


In [ ]:
# Corollary 2.4 in Mohammadi 2014 - for multi-d  - taken from Simsekli et al. ICML 2019.
def alpha_estimator_multi(m, X):
    # X is N by d matrix
    N = len(X)
    n = int(N/m) # must be an integer
    Y = torch.sum(X.view(n, m, -1), 1)
    eps = np.spacing(1)
    Y_log_norm = torch.log(Y.norm(dim=1) + eps).mean()
    X_log_norm = torch.log(X.norm(dim=1) + eps).mean()
    diff = (Y_log_norm - X_log_norm) / math.log(m)
    return 1 / diff.item()

In [ ]:
# total number of data points 
n = 10000

# dimension
d = 100

In [ ]:
# Synthetic Data Generation
sig_A = 1.0
sig_x = 3
sig_Y = 3

# explanatory vars.
A = sig_A * np.random.randn(n,d)

# true regressor
x0 = sig_x * np.random.randn(d)

# responses
Y = np.dot(A,x0) + sig_Y * np.random.randn(n)

# Mode of the distribution
x_star = np.dot(np.linalg.inv(np.dot(A.T,A)), np.dot(A.T,Y))


In [ ]:
# Loss function
def loss_fn(x,A,y,nsamp):
    res = np.sum((np.dot(A,x) - y)**2)/(2*nsamp)
    return res

In [ ]:
def sgd_linreg(A,Y,n,d,K,eta,b, compute_loss=False):
    burn_in = int(K*0.5)
    loss_sgd = np.zeros(K)

    x_cur = 5 * np.random.randn(d)
    x_mc  = np.zeros(d)
    
    if(compute_loss is True):
        loss_sgd[0] = loss_fn(x_cur,A,Y,n)

    for k in range(1,K):
        ix = np.random.permutation(n)
        S_k = ix[0:int(b)]
        A_k = A[S_k,:]
        y_k = Y[S_k]
        
        grad = np.dot(A_k.T, np.dot(A_k,x_cur)) - np.dot(A_k.T, y_k)
        grad = grad / b
        x_cur = x_cur - eta * grad
        
        if(k >= burn_in):
            k_loc = (k - burn_in) * 1.0
            x_mc  = (k_loc/(k_loc+1)) * x_mc + 1/(k_loc+1)*x_cur

        if(compute_loss is True):
            loss_sgd[k] = loss_fn(x_cur,A,Y,n)

    x_final = x_cur
    if(compute_loss is True):
        plt.figure()
        plt.plot(loss_sgd)
        
    return [x_final,x_mc]

In [ ]:
### SGD

# Numer of repetitions
m = 40
numRep = m*m

# Number of iteratios
K = 1000

#batch size
b_all   = [1, 2,3,4,5,10,20]
#step-size
eta_all = np.linspace(0.02,0.2,10)

alpha_mc_sas     = np.zeros((len(b_all), len(eta_all)))

for bi, b in enumerate(b_all):
    for ei, eta in enumerate(eta_all):
        print('b =',b,' eta =',eta)
        
        x_final = np.zeros((numRep,d))
        x_mc = np.zeros((numRep,d))

        tmp_res = Parallel(n_jobs=10)(delayed(sgd_linreg)(A,Y,n,d,K,eta,b) for i in range(numRep))
        tmp_res = np.asarray(tmp_res)
        
        x_final = np.squeeze(tmp_res[:,0,:])
        x_mc    = np.squeeze(tmp_res[:,1,:])
        
        Xfn = x_final - x_star
        Xmc = x_mc - x_star
         
        alp_tmp = [alpha_estimator_multi(mm, torch.from_numpy(Xmc)) for mm in (2, 5, 10, 20, 50, 100, m)] 
        alpha_mc_sas [bi,ei] = np.median(alp_tmp)
        
        print('\t',alp_tmp)
